# 使用`CPM`模型和`TNEWS`数据集进行零样本文本分类

1. 加载TNEWS数据集
2. 分词
3. 计算困惑度
4. 计算准确率
5. 零样本分类

## 1 加载TNEWS数据集

In [1]:
from datasets import load_dataset

tnews_ds = load_dataset("clue/clue", "tnews")
valid_set = tnews_ds["validation"]
valid_set

README.md: 0.00B [00:00, ?B/s]

tnews/test-00000-of-00001.parquet:   0%|          | 0.00/655k [00:00<?, ?B/s]

tnews/train-00000-of-00001.parquet:   0%|          | 0.00/3.40M [00:00<?, ?B/s]

tnews/validation-00000-of-00001.parquet:   0%|          | 0.00/643k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/53360 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 10000
})

In [2]:
label_names = valid_set.features["label"].names
label_indices = [i for i in range(len(label_names))]
print(label_indices)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


## 2 分词

In [3]:
ckpt = "TsinghuaAI/CPM-Generate"

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(ckpt)

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [5]:
def preprocess(sample, tokenizer):
    tokenized = tokenizer(
        sample["sentence"],
        truncation=True
    )

In [6]:
dataset = valid_set.map(
    preprocess,
    batched=True,
    fn_kwargs={"tokenizer": tokenizer}
)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.628 seconds.
Prefix dict has been built successfully.


In [7]:
dataset[0]

{'sentence': '江疏影甜甜圈自拍，迷之角度竟这么好看，美吸引一切事物', 'label': 2, 'idx': 0}

## 3 计算困惑度

In [8]:
!pip install --quiet evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == 

In [9]:
import evaluate

2025-08-14 11:26:38.474746: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755170798.641120      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755170798.698632      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
import numpy as np

In [11]:
def compute_ppl(sample, ppl_metric, model_id):
    sent, true_label = sample["sentence"], sample["label"]
    true_labels = [true_label]
    false_labels = np.random.choice(
        [k for k in label_indices if k != true_label],
        3,
        replace=False
    ).tolist()
    candidate_labels = true_labels + false_labels
    prompts = []
    for label in candidate_labels:
        prompt = f"这是关于{label}的文章: {sent}"  # 论文给出的模板
        prompts.append(prompt)
    results = ppl_metric.compute(predictions=prompts, model_id=model_id)
    pred_label = candidate_labels[np.argmin(results["perplexities"])]
    return pred_label

In [12]:
ppl_metric = evaluate.load("perplexity", module="metric")

In [13]:
pred_label = compute_ppl(dataset[0], ppl_metric, ckpt)

pytorch_model.bin:   0%|          | 0.00/10.4G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/10.4G [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
print(dataset[0])
print(pred_label)

{'sentence': '江疏影甜甜圈自拍，迷之角度竟这么好看，美吸引一切事物', 'label': 2, 'idx': 0}
1


## 4 计算准确率

In [15]:
def compute_accuracy(predicted_label, true_label, acc_metric):
    acc_metric.add(predictions=predicted_label, references=true_label)

In [16]:
acc_metric = evaluate.load("accuracy", module="metric")

In [17]:
true_label = dataset[0]["label"]
compute_accuracy(pred_label, true_label, acc_metric)
acc = acc_metric.compute()
print(acc)

{'accuracy': 0.0}


## 5 零样本分类

In [18]:
ckpt

'TsinghuaAI/CPM-Generate'

In [19]:
def classify_by_zero_shot(sample, ppl_metric, acc_metric, model_id):
    pred_label = compute_ppl(sample, ppl_metric, model_id=ckpt)
    true_label = sample["label"]
    compute_accuracy(pred_label, true_label, acc_metric)

### 5.1 使用小样本进行零样本分类

#### 5.1.1 并行运行

In [20]:
from datasets import Dataset

In [21]:
subset = Dataset.from_dict(dataset[:2])
subset

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 2
})

In [22]:
# ppl_metric = evaluate.load("perplexity", module="metric")
# acc_metric = evaluate.load("accuracy", module="metric")
# subset.map(
#     classify_by_zero_shot,
#     batched=True,
#     fn_kwargs={
#         "ppl_metric": ppl_metric,
#         "acc_metric": acc_metric,
#         "model_id": ckpt
#     }
# )

In [23]:
# acc_metric.compute()

#### 5.1.2 迭代执行

In [24]:
from tqdm import tqdm

In [25]:
# ppl_metric = evaluate.load("perplexity")
# acc_metric = evaluate.load("accuracy")
# for sample in tqdm(subset, desc="Classifing"):
#     classify_by_zero_shot(sample, ppl_metric, acc_metric, ckpt)
# subset_results = acc_metric.compute()
# subset_results

### 5.2 使用整个验证集进行零样本分类

In [26]:
# ppl_metric = evaluate.load("perplexity", module="metric")
# acc_metric = evaluate.load("accuracy", module="metric")
# dataset.map(
#     classify_by_zero_shot,
#     batched=True,
#     fn_kwargs={
#         "ppl_metric": ppl_metric,
#         "acc_metric": acc_metric,
#         "model_id": ckpt
#     }
# )
# acc_results = acc_metric.compute()

In [29]:
ppl_metric = evaluate.load("perplexity")
acc_metric = evaluate.load("accuracy")

count = 0
try:
    for sample in tqdm(dataset, desc="Classifing"):
        classify_by_zero_shot(sample, ppl_metric, acc_metric, ckpt)
        count += 1
except Exception:
    print(f"current sample index is: {count}")

Classifing:   0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Classifing:   0%|          | 1/10000 [00:09<26:57:47,  9.71s/it]

  0%|          | 0/1 [00:00<?, ?it/s]

Classifing:   0%|          | 2/10000 [00:17<24:16:39,  8.74s/it]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Classifing:   2%|▏         | 191/10000 [24:25<20:41:39,  7.60s/it]

  0%|          | 0/1 [00:00<?, ?it/s]

Classifing:   2%|▏         | 192/10000 [24:33<20:54:28,  7.67s/it]


KeyboardInterrupt: 

In [30]:
dataset_results = acc_metric.compute()
dataset_results

{'accuracy': 0.22395833333333334}

In [31]:
count

192